## import Python packages

In [2]:
from lyricsgenius import Genius
import pandas as pd
import requests
from itertools import chain
from collections import Counter
from string import punctuation
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import re
tbl = str.maketrans({ord(ch):" " for ch in punctuation})

## authenticate Genius API

In [2]:
token  = 'zvYH5XVlo2URyfxjtHK-4aypOPl63Dved-diSY-FK82a6hAOgq7yNUIsrG1dchEC'
genius = Genius(token)

## authenticate Spotify API

In [3]:
cid  ='4bbdc8c4be9f415080b262deb3e33c16'
secret ='8b4a63460af64e30b702d27c1dafda4f'

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
## Below takes in all the different LVMH brands

In [5]:
lvmh = ['birkin','henny','hennessy','dior','louis','lv','hermes','moet','louis vuitton','vuitton','dom pérignon',
       'dom','perignon','hublot','givenchy','fendi','fenty','tiffany','christian','christian dior','marc',
        'bulgari','marc jacob','jacob']

In [6]:
### Below function pulls the lyrics of each song and counts the frequency of how many times a LVMH brand is 
### mentioned in the song's lyrics

In [7]:
def lyric_counter(song_df,word_list):
    total_counter = {}
    lyrics = song_df['lyrics']
    artist = song_df['artist']
    for i in range(len(lyrics)):
        print(album_df['song'][i])
        lyric = lyrics[i]
        count = Counter(lyric.lower().translate(tbl).split())
        d =  {key.lower(): count[key] for key in count if key in word_list}
        total_counter = dict(Counter(d)+Counter(total_counter))
    return total_counter
    

### Below function takes a Spotify playlist and a given year, pulls the song, artist and song ID from Spotify. 
### Using the song and artist, it uses Genius API to search for the song and its corresponding meta data. Both the
### Spotify and Genius API is saved in the dataframe and then output to a .csv file 

In [ ]:
def get_songs(playlist,year):
    song_row = pd.DataFrame()
    playlist = sp.playlist(playlist)
    song_df = pd.DataFrame()
    for track in range(len(playlist['tracks']['items'])):
        try:
            print(track)
            song_name = playlist['tracks']['items'][track]['track']['name']
            sp_artist = playlist['tracks']['items'][track]['track']['artists'][0]['name']
            sp_song_id = playlist['tracks']['items'][track]['track']['id']
            sp_artist_id = playlist['tracks']['items'][track]['track']['artists'][0]['id']
            song_name = re.sub(r"\([^()]*\)", "", song_name)
            song_search = genius.search_song(song_name, sp_artist)

            song_json = song_search.to_json()
            song_dict = json.loads(song_json)
            time.sleep(1)
            song_id = song_dict['id']
            df_artist = sp_artist
            song = song_name
            lyrics = genius.lyrics(song_id)
            song_id = str(song_dict['id'])
            song_df = song_df.append(
            {'rtist': song_search.artist,
             'year':year,
            'song' : song_name,
            'Genius artist_id':str(song_dict['primary_artist']['id']),
            'Genius song_id': song_id,
             'Spotify song_id': sp_song_id,
             'Spotify artist_id': sp_artist_id,
            'lyrics': lyrics},
            ignore_index=True
            )
        except:
            pass
        
    file_name = year+'rap_lyrics_database.csv'
    song_df.to_csv(file_name)
    print(f'Completed storing all lyrics for {year}')
    return song_df    

In [ ]:
### Creates a Pandas dataframe for the counts of LVMH brands in lyrics

In [9]:
def create_counts(df,year):
    counts = lyric_counter(df,lvmh)
    counts_df = pd.DataFrame.from_dict(counts,orient='index').rename(columns ={0:year})
    return counts_df

In [8]:
### Python dictionary with all of years and Spotify URIs of Spotify Playlist of top Rap songs for that respective
### year

In [ ]:
playlists = {'2016':'6hAQqJYxe5mt9jWQjjwZde',
             '2017':'5ZXRYuiSQZeQKRhDFSgO3b',
             '2018':'5FyRUTsjmvQqBZnnnjFUmq',
             '2019':'1BCCebMPU7qCgEg8oy0cq4',
             '2020':'6Pw0qVok3XrfyER6ZvFD5V',
}

In [ ]:
### Loop through each year & playlist, and count LVMH brands for each song, output the count dataframe to a .csv file

In [ ]:
for year, playlist in playlists.items():
    try:
        songs_all = get_songs(playlist,year)
        counts_all = create_counts(songs_all,year)
        count_filename = year+'lvmh_counts_database.csv'
        counts_all.to_csv(count_filename)
        print(f'Completed all counts for {year}')
    except TypeError:
        print(f'Error found in {year}, skipping to next')
        time.sleep(5)

In [ ]:
### Read all the .csv files into Pandas dataframes

In [ ]:
counts_2020 = pd.read_csv('2020lvmh_counts_database.csv')
counts_2019 = pd.read_csv('2019lvmh_counts_database.csv')
counts_2018 = pd.read_csv('2018lvmh_counts_database.csv')
counts_2017 = pd.read_csv('2017lvmh_counts_database.csv')
counts_2016 = pd.read_csv('2016lvmh_counts_database.csv')
counts_2015 = pd.read_csv('2015lvmh_counts_database.csv')


In [ ]:
### Merge all of them to one big dataframe

In [ ]:
counts = counts_2020.merge(counts_2019,how='outer')
counts = counts.merge(counts_2018,how='outer')
counts = counts.merge(counts_2017,how='outer')
counts = counts.merge(counts_2016,how='outer')
counts = counts.merge(counts_2015,how='outer')

In [ ]:
### Rename columns into years and brands

In [ ]:
counts.rename(columns={'Unnamed: 0':'Brands'},inplace=True)

In [ ]:
counts['Total']= counts.sum(axis=1,numeric_only=True)
counts = counts[['Brands','2015','2016','2017','2018','2019','2020','Total']]
counts = counts.sort_values(by='Total',ascending=False)
counts = counts.set_index('Brands')
counts

In [9]:
### Aggregate counts of any brands with multiple names(i.e henny and hennessy) into one

In [ ]:
hennessy = counts.loc['henny'] + counts.loc['hennessy']
louisv = counts.loc['louis'] + counts.loc['vuitton']
christiandior = counts.loc['christian'] + counts.loc['dior']
marcjacobs = counts.loc['marc'] + counts.loc['jacob']
hennessy.name = 'henny/hennessy'
louisv.name = 'louis vuitton'
christiandior.name = 'christian dior'
marcjacobs.name = 'marc jacobs'
counts = counts.append([hennessy])
counts = counts.append([louisv])
counts = counts.append([christiandior])
counts = counts.append([marcjacobs])

In [ ]:
### Sort by total

In [ ]:
counts = counts.sort_values(by='Total',ascending=False)
counts

In [10]:
## Next step: Now that we have all rap songs, their artists, and the frequency of using LVMH brands, next step is
## to see which of these brands are being streamed the most using Spotify's Chart data from spotifychart.com

In [11]:
### Download all the Spotify streaming data and merge it into one big dataframe

In [ ]:
import os
path = '/Users/Arturo/Desktop/Cacicazgo/Code/Genius-API/spotify_streams'
file_list = os.listdir(path) 
final_df = pd.DataFrame()
for file in file_list:
    newpath = os.path.join(path, file)
    df = pd.read_csv(newpath)
    print(newpath)
    final_df = final_df.append(df)

In [ ]:
### Rename columns 

In [ ]:
final_df= final_df.rename(columns = {'Unnamed: 0':'Position','Unnamed: 1':'Song Name','Unnamed: 2':'Artist','Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.':'Streams',
                                     'Unnamed: 4':'URL'})
final_df = final_df[1:]
final_df = final_df[~final_df['Streams'].isin(['Streams'])]
final_df



In [1]:
## Aggregate all the streams since 2017 and group them by song, artist, and Spotify URI

In [ ]:
streams = final_df.groupby(['Song Name','Artist','URL']).agg(
    {
    'Streams':sum
    }
).sort_values(by='Streams',ascending=False)
streams.to_csv('spotify_streams_2017_to_02_26_2021.csv')

In [11]:
streams_final_table = pd.read_csv('spotify_streams_2017_to_02_26_2021.csv')
URL_list = streams_final_table['URL'].tolist()
lyrics_df = pd.DataFrame()
for i in URL_list:
    try:
        print(i)
        track_info = sp.track(i)
        sp_song_id = track_info['id']
        sp_artist_id = track_info['artists'][0]['id']
        song_name = track_info['name']
        sp_artist = track_info['artists'][0]['name']
        song_name = re.sub(r"\([^()]*\)", "", song_name)
 

        song_search = genius.search_song(song_name,sp_artist)
        song_json = song_search.to_json()
        song_dict = json.loads(song_json)
        time.sleep(1)
        genius_song_id = song_dict['id']
        genius_artist_id = str(song_dict['primary_artist']['id'])
        lyrics = genius.lyrics(genius_song_id)

        genius_song_id = str(genius_song_id)
        lyrics_df = lyrics_df.append(
                {'artist': song_search.artist,
                'song' : song_name,
                'Genius artist_id':genius_artist_id,
                'Genius song_id': genius_song_id,
                 'Spotify song_id': sp_song_id,
                 'Spotify artist_id': sp_artist_id,
                'lyrics': lyrics},
                ignore_index=True
                )
        
   
    except:
        print(f'Did not add lyrics to {song_name}')
        pass

https://open.spotify.com/track/3ee8Jmje8o58CHK66QrVC2
Searching for "SAD!" by XXXTENTACION...
Done.
https://open.spotify.com/track/2xLMifQCjDGFmkHkpNLD9h
Searching for "SICKO MODE" by Travis Scott...
Done.
https://open.spotify.com/track/6gBFPUFcJLzWGx4lenP6h2
Searching for "goosebumps" by Travis Scott...
Done.
https://open.spotify.com/track/7m9OqQk4RVRkw9JJdeAw96
Searching for "Jocelyn Flores" by XXXTENTACION...
Done.
https://open.spotify.com/track/7KXjTSCq5nL1LoYtL7XAwS
Searching for "HUMBLE." by Kendrick Lamar...
Done.
https://open.spotify.com/track/75ZvA4QfFiZvzhj2xkaWAh
Searching for "I Fall Apart" by Post Malone...
Done.
https://open.spotify.com/track/3a1lNhkSLSkpJE4MSHpDu9
Searching for "Congratulations" by Post Malone...
Done.
https://open.spotify.com/track/7dt6x5M1jzdTEt8oCbisTK
Searching for "Better Now" by Post Malone...
Done.
https://open.spotify.com/track/7GX5flRQZVHRAGd6B4TmDO
Searching for "XO Tour Llif3" by Lil Uzi Vert...
Done.
https://open.spotify.com/track/0nbXyq5TXYP

Done.
https://open.spotify.com/track/2G7V7zsVDxg1yRsu7Ew9RJ
Searching for "In My Feelings" by Drake...
Done.
https://open.spotify.com/track/7aiClxsDWFRQ0Kzk5KI5ku
Searching for "hot girl bummer" by blackbear...
Done.
https://open.spotify.com/track/3jjujdWJ72nww5eGnfs2E7
Searching for "Adore You" by Harry Styles...
Done.
https://open.spotify.com/track/2fQrGHiQOvpL9UgPvtYy6G
Searching for "Bank Account" by 21 Savage...
Done.
https://open.spotify.com/track/0SGkqnVQo9KPytSri1H6cF
Searching for "Bounce Back" by Big Sean...
Done.
https://open.spotify.com/track/45Egmo7icyopuzJN0oMEdk
Searching for "Love Lies " by Khalid...
Done.
https://open.spotify.com/track/7wGoVu4Dady5GV0Sv4UIsx
Searching for "rockstar" by Post Malone...
Done.
https://open.spotify.com/track/3B54sVLJ402zGa6Xm4YGNe
Searching for "Unforgettable" by French Montana...
Done.
https://open.spotify.com/track/2iUXsYOEPhVqEBwsqP70rE
Searching for "Youngblood" by 5 Seconds of Summer...
Done.
https://open.spotify.com/track/1yxgsra98r3q

Done.
https://open.spotify.com/track/4e4fqjx0Izh4svvTef1z7e
Searching for "Meant to Be " by Bebe Rexha...
Done.
https://open.spotify.com/track/5yY9lUy8nbvjM1Uyo1Uqoc
Searching for "Life Is Good " by Future...
Done.
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
Searching for "THE SCOTTS" by THE SCOTTS...
Done.
https://open.spotify.com/track/3GCdLUSnKSMJhs4Tj6CV3s
Searching for "All The Stars " by Kendrick Lamar...
Done.
https://open.spotify.com/track/2kJwzbxV2ppxnQoYw4GLBZ
Searching for "If the World Was Ending - feat. Julia Michaels" by JP Saxe...
Done.
https://open.spotify.com/track/1e1JKLEDKP7hEQzJfNAgPl
Searching for "Magnolia" by Playboi Carti...
Done.
https://open.spotify.com/track/2P91MQbaiQOfbiz9VqhqKQ
Searching for "I Like Me Better" by Lauv...
Done.
https://open.spotify.com/track/6eT7xZZlB2mwyzJ2sUKG6w
Searching for "White Iverson" by Post Malone...
Done.
https://open.spotify.com/track/51rXHuKN8Loc4sUlKPODgH
Searching for "King's Dead " by Jay Rock...
Done.
https://ope

Done.
https://open.spotify.com/track/5AEtlRudpgdT5FtNiuly6Y
Searching for "Let Me Go " by Hailee Steinfeld...
Done.
https://open.spotify.com/track/0sf12qNH5qcw8qpgymFOqD
Searching for "Blinding Lights" by The Weeknd...
Done.
https://open.spotify.com/track/7wBJfHzpfI3032CSD7CE2m
Searching for "STARGAZING" by Travis Scott...
Done.
https://open.spotify.com/track/6WrI0LAC5M1Rw2MnX2ZvEg
Searching for "Don't Start Now" by Dua Lipa...
Done.
https://open.spotify.com/track/42CeaId2XNlxugDvyqHfDf
Searching for "Candy Paint" by Post Malone...
Done.
https://open.spotify.com/track/6H0AwSQ20mo62jGlPGB8S6
Searching for "It's A Vibe" by 2 Chainz...
Done.
https://open.spotify.com/track/1K5KBOgreBi5fkEHvg5ap3
Searching for "Life Is Good " by Future...
Done.
https://open.spotify.com/track/0d2iYfpKoM0QCKvcLCkBao
Searching for "Eastside " by benny blanco...
Done.
https://open.spotify.com/track/0RLLvYBopDwooCyvnTwgYu
Searching for "F.N" by Lil Tjay...
Done.
https://open.spotify.com/track/5GXAXm5YOmYT0kL5jHv

Done.
https://open.spotify.com/track/1HXy5I3HTWq8OvxCn0z7G7
Searching for "Heaven" by Kane Brown...
Done.
https://open.spotify.com/track/3nS9a01VvXHQriLqJYwRqG
Searching for "DOLLAZ ON MY HEAD " by Gunna...
Done.
https://open.spotify.com/track/4aWmUDTfIPGksMNLV2rQP2
Searching for "Despacito " by Luis Fonsi...
Done.
https://open.spotify.com/track/1mXVgsBdtIVeCLJnSnmtdV
Searching for "Too Good At Goodbyes" by Sam Smith...
Done.
https://open.spotify.com/track/4hPpVbbakQNv8YTHYaOJP4
Searching for "One Thing Right" by Marshmello...
Done.
https://open.spotify.com/track/3Q3myFA7q4Op95DOpHplaY
Searching for "do re mi" by blackbear...
Done.
https://open.spotify.com/track/2qT1uLXPVPzGgFOx4jtEuo
Searching for "no tears left to cry" by Ariana Grande...
Done.
https://open.spotify.com/track/59J5nzL1KniFHnU120dQzt
Searching for "wokeuplikethis*" by Playboi Carti...
Done.
https://open.spotify.com/track/5274I4mUMnYczyeXkGDWZN
Searching for "Fine China" by Future...
Done.
https://open.spotify.com/track/

...

Done.
https://open.spotify.com/track/6cX1iTffMIAU8lEWSOYIIO
Searching for "Leanin'" by Lil Peep...
Done.
https://open.spotify.com/track/4NSW0Km5ZG60L8FthUebPJ
Searching for "What I Like About You " by Jonas Blue...
Done.
https://open.spotify.com/track/0HC6S4VpCGAZvyxTdrMRIQ
Searching for "De Una Vez" by Selena Gomez...
Done.
https://open.spotify.com/track/0p3tGNjc5enjSCnH1GljiH
Searching for "Me Likey" by Trippie Redd...
Done.
https://open.spotify.com/track/1s71G3V7XQaVo5IOR6gLbp
Searching for "homecoming queen?" by Kelsea Ballerini...
Done.
https://open.spotify.com/track/3SGgeDQ3cAuNSFb03PjXtv
Searching for "ADDYS " by Gunna...
Done.
https://open.spotify.com/track/13TMuxC8tdSfthXpHG9D26
Searching for "1 SIDED LOVE" by blackbear...
Done.
https://open.spotify.com/track/7nD5MiXspW533xaREDma2N
Searching for "Buck 50" by DJ Scheme...
Done.
https://open.spotify.com/track/4eZKAB9uNMIG0yNri34PBp
Searching for "Bussin " by Blueface...
Done.
https://open.spotify.com/track/54QmQd9Nc2EUDha6dD8V9X

Did not add lyrics to Long Time - Intro
https://open.spotify.com/track/3HNWVQHXpW70ubsDhvnegu
Searching for "The Bigger Picture" by Lil Baby...
Done.
https://open.spotify.com/track/17JYzDCW1UTndfURuTydF2
Searching for "Dance To This " by Troye Sivan...
Done.
https://open.spotify.com/track/1883mCuSqnfnrWoq8i3myH
Searching for "Programs" by Mac Miller...
Done.
https://open.spotify.com/track/0tQmgwFKw9069z1BXniOiA
Searching for "Provide " by G-Eazy...
Done.
https://open.spotify.com/track/1l9G7M8gNyQsgOGWZGoQsT
Searching for "Look What I Found" by Lady Gaga...
Done.
https://open.spotify.com/track/3i01Zk9c0FQ5HA1WJjflXI
Searching for "Kung Fu" by Cordae...
Done.
https://open.spotify.com/track/6bCWyr3C3ngZdwizTYWQTG
Searching for "Onna Come Up  [Remix]" by Lil Eazzyy...
Done.
https://open.spotify.com/track/7JwmlNz8ic9ATnLn7DyAKx
Searching for "I Don’t Wanna Break Up" by Future...
Done.
https://open.spotify.com/track/09L4qpS3qvz725z6K8yW0M
Searching for "Glow Up" by NAV...
Did not add lyrics 

Done.
https://open.spotify.com/track/1iDPB2WNsGEWuKycQyDGn6
Searching for "Crazy" by Lil Pump...
Done.
https://open.spotify.com/track/2c09gumRCmu3qmbcqdQbUN
Searching for "THUG LIFE" by BROCKHAMPTON...
Done.
https://open.spotify.com/track/5lyikKk6KFWfWCuDDir9Fx
Searching for "Rockin' Around The Christmas Tree" by Miley Cyrus...
Done.
https://open.spotify.com/track/4J2RaG4UH2HCThcqzDpeAF
Searching for "Start Over" by Imagine Dragons...
Done.
https://open.spotify.com/track/2IjgKCbrk8CtqmsOC5DhVd
Searching for "Fuck That Check Up " by Meek Mill...
Done.
https://open.spotify.com/track/7DHNcoR73yUYCSn6PqBgkH
Searching for "10 F*CKS " by Tory Lanez...
No results found for: '10 F*CKS  Tory Lanez'
Did not add lyrics to 10 F*CKS 
https://open.spotify.com/track/4X4cPQicoAfannkUFUXmi9
Searching for "That Go! " by Young Stoner Life...
Done.
https://open.spotify.com/track/5eGX87IiKsGuzS3iw4CfCX
Searching for "September - Recorded at The Tracking Room Nashville" by Taylor Swift...
Done.
https://open

Done.
https://open.spotify.com/track/4OEOEsOhlzDqQuewiuPJAc
Searching for "Whiskey " by Maroon 5...
Done.
https://open.spotify.com/track/0zcmVx57BsnlYaPu7k2e2e
Searching for "Whole Lot" by 21 Savage...
Done.
https://open.spotify.com/track/6Dwtha2FtZFoMEBh5GR2sq
Searching for "CRZY" by Kehlani...
Done.
https://open.spotify.com/track/1c5Lbn9mYwNYbnNiizw6D0
Searching for "Iced Out " by Lil Pump...
Done.
https://open.spotify.com/track/2RCp9QBnFSxdKe8RD4xF8s
Searching for "Momentz " by Gorillaz...
Done.
https://open.spotify.com/track/2c5D6B8oXAwc6easamdgVA
Searching for "Don't Sleep on Me " by Ty Dolla $ign...
Done.
https://open.spotify.com/track/5uMVjsPAVOc8v3iTgVkkrz
Searching for "Church" by Fall Out Boy...
Done.
https://open.spotify.com/track/7w9bgPAmPTtrkt2v16QWvQ
Searching for "Lose Yourself - From "8 Mile" Soundtrack" by Eminem...
Done.
https://open.spotify.com/track/5xS9hkTGfxqXyxX6wWWTt4
Searching for "Mary Jane's Last Dance" by Tom Petty and the Heartbreakers...
Done.
https://open

Done.
https://open.spotify.com/track/4KVVCo25PROevpOdHyvYRk
Searching for "Teach Me a Lesson" by Bryson Tiller...
Done.
https://open.spotify.com/track/3LRoXCTQ7VvvVtzpceHt4y
Searching for "Dead Presidents " by Rick Ross...
Done.
https://open.spotify.com/track/06u5LrUpbosQlQ1QJFhPpG
Searching for "Drew Barrymore" by SZA...
Done.
https://open.spotify.com/track/7DEgL4rErTwjrsxLMlUVhf
Searching for "Any Ol' Barstool" by Jason Aldean...
Done.
https://open.spotify.com/track/650OeHTLxZAQmb4aEbGmaA
Searching for "Good Goodbye " by Linkin Park...
Done.
https://open.spotify.com/track/0IqKeD8ZSP72KbGYyzEcAs
Searching for "Take Me To Church" by Hozier...
Done.
https://open.spotify.com/track/23TxRN09aR1RB0G0tFoT0b
Searching for "Better Man" by Little Big Town...
Done.
https://open.spotify.com/track/5IKOXYnhzrLxqhlI0btiaa
Searching for "Red Roses " by Lil Skies...
Done.
https://open.spotify.com/track/1nxufatvWIGPPBtPACXzu8
Searching for "Let 'Em Talk " by Kesha...
Done.
https://open.spotify.com/trac

In [ ]:
### Merge the streams table with the lyrics table on the song name

In [12]:
final_stream_lyrics_table = streams_final_table.merge(lyrics_df,how='inner',left_on='Song Name',right_on='song')

In [13]:
final_stream_table = final_stream_lyrics_table.drop_duplicates(subset=['Song Name']).reset_index()
final_stream_table

,index,Song Name,Artist,URL,Streams,Genius artist_id,Genius song_id,Spotify artist_id,Spotify song_id,artist,lyrics,song
0,0,SAD!,XXXTENTACION,https://open.spotify.com/track/3ee8Jmje8o58CHK...,577901267.0,396565,3430186,15UsOTVnJzReFVN1VCnxy4,3ee8Jmje8o58CHK66QrVC2,XXXTENTACION,[Intro]\nYeah\n\n[Chorus]\nWho am I? Someone t...,SAD!
1,4,SICKO MODE,Travis Scott,https://open.spotify.com/track/2xLMifQCjDGFmkH...,550670975.0,20185,3876994,0Y5tJX1MQlPlqiwlOH1tJY,2xLMifQCjDGFmkHkpNLD9h,Travis Scott,"[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun i...",SICKO MODE
2,5,goosebumps,Travis Scott,https://open.spotify.com/track/6gBFPUFcJLzWGx4...,549870433.0,20185,2849767,0Y5tJX1MQlPlqiwlOH1tJY,6gBFPUFcJLzWGx4lenP6h2,Travis Scott,[Intro: Travis Scott]\nYeah\n7:30 in the night...,goosebumps
3,6,Jocelyn Flores,XXXTENTACION,https://open.spotify.com/track/7m9OqQk4RVRkw9J...,532330080.0,396565,3185661,15UsOTVnJzReFVN1VCnxy4,7m9OqQk4RVRkw9JJdeAw96,XXXTENTACION,"[Intro: Shiloh Dynasty]\nI know you so well, s...",Jocelyn Flores
4,7,HUMBLE.,Kendrick Lamar,https://open.spotify.com/track/7KXjTSCq5nL1LoY...,527892188.0,1421,3039923,2YZyLoL8N0Wb9xBt1NhZWg,7KXjTSCq5nL1LoYtL7XAwS,Kendrick Lamar,[Intro]\nNobody pray for me\nIt been that day ...,HUMBLE.
...,...,...,...,...,...,...,...,...,...,...,...,...
2839,5188,Middle,DJ Snake,https://open.spotify.com/track/4OZEu0Vwq0RB2LA...,1037663.0,41287,2331497,540vIaP2JwjQb9dm3aArA4,4OZEu0Vwq0RB2LAq14v99n,DJ Snake,[Verse 1]\nStarin' at two different views on y...,Middle
2840,5189,Wake Up Where You Are,State of Sound,https://open.spotify.com/track/2s76aKBWKShlwPC...,1029544.0,1005514,2876087,05o1zW8b6PV6rEquXr74sB,2s76aKBWKShlwPC4xTtEEE,State of Sound,[Verse 1]\nNever thought that I would love aga...,Wake Up Where You Are
2841,5190,Nothing Without You,The Weeknd,https://open.spotify.com/track/3Te3gG6MPEDNqLG...,1017468.0,2358,2911563,1Xyo4u8uXC1ZmMpatF05PJ,3Te3gG6MPEDNqLGzWrjKC3,The Weeknd,"[Verse 1]\nI realized, I belong to you\nI feel...",Nothing Without You
2842,5191,A Lonely Night,The Weeknd,https://open.spotify.com/track/4UknpXoLjlB7Mao...,994402.0,2358,2917435,1Xyo4u8uXC1ZmMpatF05PJ,4UknpXoLjlB7Mao4UWcJ2o,The Weeknd,"[Intro]\nGassed up, yeah, yeah\nHey, na-na-na-...",A Lonely Night


In [ ]:
## Rename all of the columns accordingly

In [32]:
final_stream_table = final_stream_table[['Song Name','Artist','URL','Streams','Genius artist_id','Genius song_id',
                                        'Spotify artist_id','Spotify song_id','artist','lyrics','song']]
final_stream_table

,Song Name,Artist,URL,Streams,Genius artist_id,Genius song_id,Spotify artist_id,Spotify song_id,artist,lyrics,song
0,SAD!,XXXTENTACION,https://open.spotify.com/track/3ee8Jmje8o58CHK...,577901267.0,396565,3430186,15UsOTVnJzReFVN1VCnxy4,3ee8Jmje8o58CHK66QrVC2,XXXTENTACION,[Intro]\nYeah\n\n[Chorus]\nWho am I? Someone t...,SAD!
1,SICKO MODE,Travis Scott,https://open.spotify.com/track/2xLMifQCjDGFmkH...,550670975.0,20185,3876994,0Y5tJX1MQlPlqiwlOH1tJY,2xLMifQCjDGFmkHkpNLD9h,Travis Scott,"[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun i...",SICKO MODE
2,goosebumps,Travis Scott,https://open.spotify.com/track/6gBFPUFcJLzWGx4...,549870433.0,20185,2849767,0Y5tJX1MQlPlqiwlOH1tJY,6gBFPUFcJLzWGx4lenP6h2,Travis Scott,[Intro: Travis Scott]\nYeah\n7:30 in the night...,goosebumps
3,Jocelyn Flores,XXXTENTACION,https://open.spotify.com/track/7m9OqQk4RVRkw9J...,532330080.0,396565,3185661,15UsOTVnJzReFVN1VCnxy4,7m9OqQk4RVRkw9JJdeAw96,XXXTENTACION,"[Intro: Shiloh Dynasty]\nI know you so well, s...",Jocelyn Flores
4,HUMBLE.,Kendrick Lamar,https://open.spotify.com/track/7KXjTSCq5nL1LoY...,527892188.0,1421,3039923,2YZyLoL8N0Wb9xBt1NhZWg,7KXjTSCq5nL1LoYtL7XAwS,Kendrick Lamar,[Intro]\nNobody pray for me\nIt been that day ...,HUMBLE.
...,...,...,...,...,...,...,...,...,...,...,...
2839,Middle,DJ Snake,https://open.spotify.com/track/4OZEu0Vwq0RB2LA...,1037663.0,41287,2331497,540vIaP2JwjQb9dm3aArA4,4OZEu0Vwq0RB2LAq14v99n,DJ Snake,[Verse 1]\nStarin' at two different views on y...,Middle
2840,Wake Up Where You Are,State of Sound,https://open.spotify.com/track/2s76aKBWKShlwPC...,1029544.0,1005514,2876087,05o1zW8b6PV6rEquXr74sB,2s76aKBWKShlwPC4xTtEEE,State of Sound,[Verse 1]\nNever thought that I would love aga...,Wake Up Where You Are
2841,Nothing Without You,The Weeknd,https://open.spotify.com/track/3Te3gG6MPEDNqLG...,1017468.0,2358,2911563,1Xyo4u8uXC1ZmMpatF05PJ,3Te3gG6MPEDNqLGzWrjKC3,The Weeknd,"[Verse 1]\nI realized, I belong to you\nI feel...",Nothing Without You
2842,A Lonely Night,The Weeknd,https://open.spotify.com/track/4UknpXoLjlB7Mao...,994402.0,2358,2917435,1Xyo4u8uXC1ZmMpatF05PJ,4UknpXoLjlB7Mao4UWcJ2o,The Weeknd,"[Intro]\nGassed up, yeah, yeah\nHey, na-na-na-...",A Lonely Night


In [ ]:
### Count which are the most common brands overall under the LVMH umbrella

In [30]:
streams_lvmh_counter = lyric_counter(final_stream_table,lvmh)
stream_counts = pd.DataFrame.from_dict(streams_lvmh_counter,orient='index').rename(columns ={0:'counts'})
stream_counts.sort_values(by='counts',ascending=False)

,counts
louis,164
dior,73
christian,59
henny,51
fendi,49
dom,48
hennessy,37
birkin,35
givenchy,27
jacob,27


In [ ]:
### Count brands for each song, taking into account their total streams

In [37]:
final_stream_df = pd.DataFrame()
for i in range(len(final_stream_table['lyrics'])):
    for brand in lvmh_new:
        
        song = final_stream_table['Song Name'][i]
        artist = final_stream_table['artist'][i]
        lyric = final_stream_table['lyrics'][i]
        streams = final_stream_table['Streams'][i]
        
        lyric_list = lyric.lower().translate(tbl).split()
        x = lyric_list.count(brand)
        
        if x > 0:
            final_stream_df = final_stream_df.append(
                {'Song':song,
                 'Artist':artist,
                 'Brand':brand,
                 'Brand Count':x,
                 'Streams':streams,
                },
                ignore_index=True
            )
            print(f'{song} by {artist} has the brand {brand} {x} times.')

SICKO MODE by Travis Scott has the brand louis 1 times.
Better Now by Post Malone has the brand henny 1 times.
Robbery by Juice WRLD has the brand hennessy 1 times.
7 rings by Ariana Grande has the brand tiffany 1 times.
Truth Hurts by Lizzo has the brand dom 3 times.
In My Feelings by Drake has the brand henny 1 times.
hot girl bummer by ​blackbear has the brand birkin 1 times.
rockstar by Post Malone has the brand henny 1 times.
EARFQUAKE by Tyler, The Creator has the brand tiffany 1 times.
Portland by Drake has the brand henny 1 times.
Mine by Bazzi has the brand hennessy 1 times.
Ransom by Lil Tecca has the brand louis 1 times.
Ransom by Lil Tecca has the brand fendi 1 times.
Ransom by Lil Tecca has the brand vuitton 1 times.
LOYALTY. FEAT. RIHANNA. by Genius Brasil Traduções has the brand dom 1 times.
VIBEZ by DaBaby has the brand louis 1 times.
One Dance by Drake has the brand hennessy 6 times.
YOSEMITE by Travis Scott has the brand louis 1 times.
Suicidal by YNW Melly has the br

Patek by Future & Lil Uzi Vert has the brand louis 1 times.
Patek by Future & Lil Uzi Vert has the brand dior 1 times.
Patek by Future & Lil Uzi Vert has the brand christian 1 times.
Bitchuary by Shordie Shordie has the brand henny 3 times.
London Boy by Taylor Swift has the brand louis 2 times.
Driving Home for Christmas - 2019 Remaster by Oliver Stone has the brand dom 2 times.
$$$ - with Matt Ox by Genius Users has the brand birkin 1 times.
CIVIL WAR - Bonus by Genius has the brand dior 2 times.
Venetia by Lil Uzi Vert has the brand louis 1 times.
Bigger Than Life by Lil Uzi Vert has the brand louis 1 times.
I'm so Groovy by Future has the brand louis 1 times.
I'm so Groovy by Future has the brand henny 1 times.
4th Dimension by KIDS SEE GHOSTS has the brand louis 4 times.
Since Way Back by Drake has the brand christian 1 times.
Rap Saved Me by 21 Savage, Offset & Metro Boomin has the brand louis 1 times.
Rap Saved Me by 21 Savage, Offset & Metro Boomin has the brand givenchy 1 time

SAD SH*T by Genius has the brand louis 1 times.
SAD SH*T by Genius has the brand jacob 1 times.
Get Ugly by Lil Baby has the brand jacob 2 times.
Biking by Frank Ocean has the brand tiffany 1 times.
Pretty Mami by Lil Uzi Vert has the brand dior 1 times.
Championships by Meek Mill has the brand dior 1 times.
Championships by Meek Mill has the brand christian 1 times.
Burberry faygo by Genius Deutschland has the brand dior 1 times.
TALK ABOUT IT by DaBaby has the brand fendi 1 times.
Unbothered by Ski Mask the Slump God has the brand henny 2 times.
Unbothered by Ski Mask the Slump God has the brand fendi 1 times.
Slide Around by Chance the Rapper has the brand fendi 1 times.
Thug Love by A Boogie wit da Hoodie has the brand louis 1 times.
Thug Love by A Boogie wit da Hoodie has the brand vuitton 1 times.
No Cap by Future & Young Thug has the brand fendi 1 times.
How U Feel by HUNCHO JACK has the brand hennessy 1 times.
Kills You Slowly by The Chainsmokers has the brand louis 1 times.
BO

In [38]:
final_stream_df

,Artist,Brand,Brand Count,Song,Streams
0,Travis Scott,louis,1.0,SICKO MODE,550670975.0
1,Post Malone,henny,1.0,Better Now,466141747.0
2,Juice WRLD,hennessy,1.0,Robbery,330679687.0
3,Ariana Grande,tiffany,1.0,7 rings,270151652.0
4,Lizzo,dom,3.0,Truth Hurts,246878029.0
...,...,...,...,...,...
370,2 Chainz,birkin,1.0,Blue Cheese,1383082.0
371,Lil Yachty,henny,2.0,X Men,1321079.0
372,Drake,givenchy,1.0,All Me,1262995.0
373,21 Savage,birkin,1.0,Special,1181094.0


In [ ]:
## Normalize all brands to have the same names in the lyrics

In [109]:
final_stream_df['Brand'] = final_stream_df['Brand'].str.capitalize()
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Henny','Hennessey')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Hennessey','Hennessey')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Hennessy','Hennessey')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Hennessey','Hennessy')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Dom','Dom Perignon')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Dom perignon','Dom Perignon')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Dior','Christian Dior')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Christian','Christian Dior')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Christian dior','Christian Dior')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Louis','Louis Vuitton')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Vuitton','Louis Vuitton')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Louis vuitton','Louis Vuitton')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('lv','Louis Vuitton')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Marc','Marc Jacob')
final_stream_df['Brand'] = final_stream_df['Brand'].replace('Jacob','Marc Jacob')


In [2]:
### Shorten the dataframe and then  groupby which brands have the most brands

In [3]:
final_stream_df = final_stream_df[['Song','Artist','Brand','Streams']]
final_stream_df

NameError: name 'final_stream_df' is not defined

In [4]:
final_stream_df.groupby(by =['Brand','Streams']).head(5)


NameError: name 'final_stream_df' is not defined

In [58]:
final_stream_df.to_csv('streams_lvmh_brands.csv')